Dan Shea  
2021-06-05  

#### Problem
A string $s$ is a supersequence of another string $t$ if $s$ contains $t$ as a subsequence.

A common supersequence of strings $s$ and $t$ is a string that serves as a supersequence of both $s$ and $t$.  
For example, "GACCTAGGAACTC" serves as a common supersequence of "ACGTC" and "ATAT". A shortest common supersequence of $s$ and $t$ is a supersequence for which there does not exist a shorter common supersequence. Continuing our example, "ACGTACT" is a shortest common supersequence of "ACGTC" and "ATAT".

__Given:__ Two DNA strings $s$ and $t$.

__Return:__ A shortest common supersequence of $s$ and $t$. If multiple solutions exist, you may output any one.

##### Sample Dataset
```
ATCTGAT
TGCATA
```
##### Sample Output
```
ATGCATGAT
```

In [1]:
import numpy as np
def compute_scores(A, B, match=1, mismatch=-1, gap=0):
    scores = np.zeros((len(A)+1, len(B)+1))
    scores[:,0] = [-i for i in range(scores.shape[0])]
    scores[0,:] = [-j for j in range(scores.shape[1])]
    for i in range(1, scores.shape[0]):
        for j in range(1, scores.shape[1]):
            if A[i-1] == B[j-1]:
                scores[i,j] = max(scores[i-1,j-1]+match, scores[i-1,j]+gap, scores[i,j-1]+gap)
            else:
                scores[i,j] = max(scores[i-1,j-1]+mismatch, scores[i-1,j]+gap, scores[i,j-1]+gap)
    return scores

def needleman_wunsch(A, B):
    if len(A) > len(B):
        A,B = B,A
    scores = compute_scores(A, B)
    i, j = len(A), len(B)
    result = []
    while (i != 0) and (j != 0):
        if A[i-1] == B[j-1]:
            i -= 1
            j -= 1
            result.append(A[i])
        else:
            i, j, s, c = max((i-1, j, scores[i-1, j], A[i-1]),
                             (i, j-1, scores[i, j-1], B[j-1]), key=lambda x: x[2])
            result.append(c)
    if i == 0:
        result += [B[j] for j in range(j-1, -1, -1)]
    elif j == 0:
        result += [A[i] for i in range(i-1, -1, -1)]
    return ''.join(reversed(result))

In [2]:
needleman_wunsch('TGCATA', 'ATCTGAT')

'ATCTGCATA'

In [3]:
needleman_wunsch('ATCTGAT', 'TGCATA')

'ATCTGCATA'

In [4]:
needleman_wunsch('ACGTC', 'ATAT')

'ACGTCAT'

In [5]:
needleman_wunsch('ATAT', 'ACGTC')

'ACGTCAT'

In [7]:
def parse_input_print_ans(filename):
    with open(filename, 'r') as fh:
        A = next(fh).strip()
        B = next(fh).strip()
        return needleman_wunsch(A, B)

In [8]:
parse_input_print_ans('sample.txt')

'ATCTGCATA'

In [9]:
parse_input_print_ans('rosalind_scsp.txt')

'CGTGACTGGGTCCTTGCATCCTCACTGACTCCAGTACGTCATCAGGCCGCAAGCTATTACGCCGTGGGAGTGGTCGAAGTCGTAGTCCCTCGCAACAGTTACGTATTCGCACGCCCTGAGGAGTA'

##### Some notes on this solution
Most descriptions of how to solve this problem will tell you that you interleave the letters of $s$ and $t$ into the longest common subsequence of $s$ and $t$. This may tempt you to re-use the previous solution for the LCSQ problem and then try and work out how to perform the interleaving. However, if we re-frame the problem as an alignment problem, we see that the global alignment of $s$ and $t$ using the Needleman-Wunsch algorithm will do this for us in one step.

The global alignment will ensure that shared symbols only appear once in the output string, and we can ensure the shortest path by using a couple of tricks when we perform the alignment back-tracking.

Right at the beginning before we do anything else we can:
- Always ensure the longer of the two strings is the 'B' by swapping them at the beginning if 'A' is logner than 'B'.

In the alignment back-tracking portion of the algorithm: 
- Ensure that we only move diagonal on matches (i.e. - do not allow mismatches)
- When the score is tied for the gapping, we always choose to move "up" because we set the "A" input to be the shorter string which defines the number of rows in the scoring matrix. This ensures we always move back towards the origin in the fewest possible steps because we consume the shorter input and then prepend any leftover nucleotides in the longer input to the beginning of the result.

When we backtrack through the scoring matrix, we consume inputs based on the direction we are moving:
- If we move diagonally, we may use either the "A" or "B" value at that position (as they are the same)
- If we move up, we use the "A" value, likewise we use the "B" value if we move left

The "alignment" is then the shortest common supersequence of the two strings $s$ and $t$.